In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
from transform_attention_weights_lib import transform_attention_weights


from visualize_attention_weights import *
%matplotlib widget

In [2]:
weights = np.load(r"E:\graphsum\data\attention_weights\pretrained_attention_weights.npy")
parent_idx = np.load(r"E:\graphsum\data\attention_weights\parent_idx.npy")
scores = np.load(r"E:\graphsum\data\attention_weights\scores.npy")
result_dict = pickle.load(open(r"E:\graphsum\data\attention_weights\save_dict" ,"rb"))

finished_scores_array = result_dict["scores_array"]

In [3]:
decoded_weight_matrix, decoded_score_matrix = transform_attention_weights(weights, parent_idx, scores, result_dict)

In [4]:
finished_scores_array[1,:,290:300]

array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [-11.98159885, -11.96436691, -12.03112888,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [-11.98159885, -11.96436691, -12.00180435, -12.22069931,
        -12.34015942, -12.50501919, -12.61776733, -12.70601749,
        -12.70512867, -12.68744755],
       [-11.98159885, -11.96436691, -12.00180435, -12.22069931,
        -12.34015942, -12.5412159 , -12.61404514, -12.71828747,
        -12.72091961, -12.70312119],
       [-11.57081032, -11.55626392, -12.18352032, -12.16792107,
        -12.15712452, -12.13747311, -12.11677933, -12.68395615,
        -12.7259407 , -12.70637798]])

In [5]:
print(decoded_score_matrix[1,:,290:300])

[[-11.57081032 -11.55626392 -12.18352032 -12.16792107 -12.15712452
  -12.13747311 -12.11677933 -12.68395615 -12.7259407  -12.70637798]
 [-11.98159885 -11.96436691 -12.00180435 -12.22069931 -12.34015942
  -12.50501919 -12.61776733 -12.70601749 -12.70512867 -12.68744755]
 [-11.98159885 -11.96436691 -12.00180435 -12.22069931 -12.34015942
  -12.5412159  -12.61404514 -12.71828747 -12.72091961 -12.70312119]
 [-11.98159885 -11.96436691 -12.03112888 -12.03112888 -12.03112888
  -12.03112888 -12.03112888 -12.03112888 -12.03112888 -12.03112888]
 [-11.01013756 -11.01013756 -11.01013756 -11.01013756 -11.01013756
  -11.01013756 -11.01013756 -11.01013756 -11.01013756 -11.01013756]]


In [6]:
finished_scores_array[0,:,200:210]

array([[-17.07486916, -17.15180016, -17.10515404,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [-16.98996162, -17.06534195, -17.42035294, -17.51957512,
        -17.47158623,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [-16.9791851 , -17.24659729, -17.42840004, -17.53141403,
        -17.48337364,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [-16.98996162, -17.06534195, -17.36968231, -17.39950562,
        -17.35833359, -17.73204041, -17.70084381, -18.14871788,
        -18.22179985, -18.29870415],
       [-16.98996162, -17.06534195, -17.36968231, -17.39950562,
        -17.35833359, -17.73204041, -17.70084381, -18.14871788,
        -18.22179985, -18.29870415]])

In [7]:
decoded_score_matrix[0,:,200:210]

array([[-16.9791851 , -17.24659729, -17.42840004, -17.53141403,
        -17.48337364, -17.48337364, -17.48337364, -17.48337364,
        -17.48337364, -17.48337364],
       [-16.98996162, -17.06534195, -17.36968231, -17.39950562,
        -17.35833359, -17.73204041, -17.70084381, -18.14871788,
        -18.22179985, -18.29870415],
       [-16.98996162, -17.06534195, -17.36968231, -17.39950562,
        -17.35833359, -17.73204041, -17.70084381, -18.14871788,
        -18.22179985, -18.29870415],
       [-16.98996162, -17.06534195, -17.42035294, -17.51957512,
        -17.47158623, -17.47158623, -17.47158623, -17.47158623,
        -17.47158623, -17.47158623],
       [-17.07486916, -17.15180016, -17.10515404, -17.10515404,
        -17.10515404, -17.10515404, -17.10515404, -17.10515404,
        -17.10515404, -17.10515404]])

In [8]:
histo_3d_simple(decoded_weight_matrix, decoded_score_matrix, (result_dict["longest_beam_array"]-1).astype("int"),decoding_layer=0,example=0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
histo_3d_multi(decoded_weight_matrix, decoded_score_matrix, (result_dict["longest_beam_array"]-1).astype("int"), decoding_layer=7)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …